<a href="https://colab.research.google.com/github/susej2/AlgoritmosOptimizacion/blob/main/Algoritmos_R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alumno: Jesus Martinez <br>
Github: https://github.com/susej2/AlgoritmosOptimizacion/
<br> **Descripcion del reto:** Implementar el algoritmo de búsqueda tabú para el TSP de la AG3.

In [10]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [11]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios
import numpy as np

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)




gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? Y


In [12]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

distancias = np.zeros((len(Nodos), len(Nodos)))

for i in range(len(Nodos)):
    for j in range(len(Nodos)):
        distancias[i][j] = problem.get_weight(Nodos[i], Nodos[j])



In [ ]:
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion

def distancia_total(solucion, distancias):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancias[solucion[i]][solucion[i + 1]]
    distancia_total += distancias[solucion[-1]][solucion[0]]
    return distancia_total


In [13]:

def buscar_solucion_tabu(distancias, tamano_lista_tabu, num_iteraciones):
    num_ciudades = len(distancias)
    mejor_solucion = crear_solucion(Nodos)
    mejor_distancia = distancia_total(mejor_solucion, distancias)
    solucion_actual = mejor_solucion.copy()

    lista_tabu = []

    for iteracion in range(num_iteraciones):
        vecina_encontrada = False
        mejor_vecina = None
        mejor_distancia_vecina = float('inf')

        for i in range(1, num_ciudades - 1):
            for j in range(i + 1, num_ciudades):
                vecina = solucion_actual.copy()
                vecina[i], vecina[j] = vecina[j], vecina[i]

                distancia_vecina = distancia_total(vecina, distancias)

                if distancia_vecina < mejor_distancia_vecina and (i, j) not in lista_tabu:
                    mejor_vecina = vecina
                    mejor_distancia_vecina = distancia_vecina
                    vecina_encontrada = True

        if vecina_encontrada:
            solucion_actual = mejor_vecina
            lista_tabu.append((i, j))
            if len(lista_tabu) > tamano_lista_tabu:
                lista_tabu.pop(0)

        if mejor_distancia_vecina < mejor_distancia:
            mejor_solucion = mejor_vecina
            mejor_distancia = mejor_distancia_vecina

    return mejor_solucion, mejor_distancia


In [14]:

solucion_optima, distancia_optima = buscar_solucion_tabu(distancias, tamano_lista_tabu=5, num_iteraciones=1000)

print("Mejor solución encontrada:", solucion_optima)
print("Distancia óptima:", distancia_optima)


Mejor solución encontrada: [0, 32, 34, 33, 20, 31, 17, 37, 15, 16, 14, 19, 13, 5, 26, 6, 3, 1, 7, 36, 35, 27, 2, 4, 18, 12, 11, 25, 10, 41, 40, 24, 21, 29, 30, 38, 22, 39, 9, 23, 8, 28]
Distancia óptima: 1635.0
